<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://cocl.us/exploring_spark_graphX">
         <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/BD0223EN/Ad/TopAd.png" width="750" align="center">
    </a>
</div>

<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/BD0223EN/Ad/CCLog.png" width="200" align="center">
</a>

# <center>Neighborhood Aggregation and Caching</center>

<img src="http://spark.apache.org/docs/latest/img/graphx_logo.png" width=600 />

## Welcome to the forth and final lab in the course, Exploring GraphX.

### GraphX is Apache Spark's API for graph and graph-parallel computations.

In this lab exercise, you will take a quick review about the graph Operators we used in the last exercise by fixing up the flaws in a graph. Then we will take a look at Neighborhood Aggregation and caching your graphs for optimized speeds. Then finally we will use real-data and compile a more complex graph!

### Some Notebook Commands
#### In case you haven't dealt with a JupyterLab before, here are some quick, useful commands that may be handy to get started.
<ul>
    <li>Run a cell: CTRL + ENTER</li>
    <li>Create a cell above a cell: a</li>
    <li>Create a cell below a cell: b</li>
    <li>Change a cell to Markdown: m</li>
    
    <li>Change a cell to code: y</li>
</ul>

<b> If you are interested in more keyboard shortcuts, go to Help -> Keyboard Shortcuts </b>

So last time we build a slightly more complex graph with A LOT of edges. You may have noticed that this graph also has a lot of problems! We are going to reconstruct the same graph and fix it up! Before we start we will have to import our usual libraries from before:

- org.apache.spark._ 
- org.apache.spark.graphx._
- org.apache.spark.rdd.RDD 

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
-->

Here is a reminder of the following attributes:

The graph will have the following Vertices: (Person)

- Billy Bill -> VertexId = 1
- Jacob Johnson -> VertexId = 2
- Stan Smith -> VertexId = 3
- Homer Simpson -> VertexId = 4
- Clark Kent -> VertexId = 5
- James Smith -> VertexId = 6

and the following Edges:

- Jacob Johnson is Friends with Billy Bill
- Jacob Johnson is Friends with Clark Kent
- Stan Smith is Friends with Billy Bill
- Stan Smith is Friends with Homer Simpson
- Stan Smith is Friends with Clark Kent
- Stan Smith is Friends with James Smith
- Homer Simpson is Friends with James Smith
- Clark Kent is Friends with Billy Bill
- Clark Kent is Friends with Billy Bill
- Clark Kent is Friends with James Smith
- James Smith is Friends with Bruce Lee (VertexId = 7)

and with the same default vertex of "Self", "Missing".

Use <code>vertexRDD</code>, <code>edgeRDD</code>, <code>defaultvertex</code>, and <code>facebook2</code> as the variables for construction.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val vertexRDD: RDD[(Long, (String, String))] = sc.parallelize(Array((1L, ("Billy Bill", "Person")), (2L, ("Jacob Johnson", "Person")), (3L, ("Stan Smith", "Person")), (4L, ("Homer Simpson", "Person")), (5L, ("Clark Kent", "Person")), (6L, ("James Smith", "Person"))))
val edgeRDD: RDD[Edge[String]] = sc.parallelize(Array(Edge(2L, 1L, "Friends"), Edge(2L, 5L, "Friends"), Edge(3L, 1L, "Friends"), Edge(3L, 4L, "Friends"), Edge(3L, 5L, "Friends"), Edge(3L, 6L, "Friends"), Edge(4L, 6L, "Friends"), Edge(5L, 1L, "Friends"), Edge(5L, 1L, "Friends"), Edge(5L, 6L, "Friends"), Edge(6L, 7L, "Friends")))
var defaultvertex = ("Self", "Missing")
var facebook = Graph(vertexRDD, edgeRDD, defaultvertex)
-->

Print to confirm!

In [ ]:
for (triplet <- facebook.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is ")
    print(triplet.attr)
    print(" with ")
    println(triplet.dstAttr._1)
}

So after everything matches, one of the first steps that we will take in fixing this graph is trying to eliminate parts of the graph that is unnecessary. Since Bruce Lee is not in our graph, we will want to eliminate the relation that "James Smith is Friends with Self." How do you think we will do this?

With the subgraph function!

We can use the subgraph function to remove the default vertex in our graph. This will also remove any edges that are connected to it, doing exact what we want!

Now create a new variable called <code>facebook_step1</code> as the new graph that is returned from the subgraph function. We are looking to remove our default vertex. Then print to confirm!

Hint: Remember, subgraph returns a subgraph that fits the conditions.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val facebook_step1 = facebook.subgraph(vpred = (id, user_type) => user_type._2 != "Missing")

for (triplet <- facebook_step1.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is ")
    print(triplet.attr)
    print(" with ")
    println(triplet.dstAttr._1)
}
-->

Awesome! So we got rid of the relationship of James with "Self." Now let's try to get rid out the duplicate relation that Clark Kent has with Billy Bill. What do you think we will use?

GroupEdges! But first we need to partition our graph with the partitionBy function with a partitionstrategy of PartitionStrategy.EdgePartition1D. Save this as <code>facebook_step2</code>.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val facebook_step2 = facebook_step1.partitionBy(PartitionStrategy.EdgePartition1D)
-->

Now try to run the groupEdges function making sure to just keep one of the two parallel edges. Save this as <code>facebook_fixed</code>. Print to confirm.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val facebook_fixed = facebook_step2.groupEdges(merge = (edge1, edge2) => edge1)

for (triplet <- facebook_fixed.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is ")
    print(triplet.attr)
    print(" with ")
    println(triplet.dstAttr._1)
}
-->

Sweet! So now we have fixed all of the visible mistakes so far. Also don't worry about the order change of how it was printed, after the partition this can change. Now given the follow graph we're accustomed with:

In [ ]:
val vertexRDD_old: RDD[(Long, (String, String))] = sc.parallelize(Array((1L, ("Billy Bill", "Person")), (2L, ("Jacob Johnson", "Person")), (3L, ("Andrew Smith", "Person")), (4L, ("Iron Man Fan Page", "Page")), (5L, ("Captain America Fan Page", "Page"))))
val edgeRDD_old: RDD[Edge[String]] = sc.parallelize(Array(Edge(1L, 2L, "Friends"), Edge(1L, 3L, "Friends"), Edge(2L, 4L, "Follower"), Edge(2L, 5L, "Follower"), Edge(3L, 5L, "Follower")))
var defaultvertex = ("Self", "Missing")
var facebook_old = Graph(vertexRDD_old, edgeRDD_old, defaultvertex)

This is the old facebook (defined as facebook_old). Here's the oh-so-familiar visualization:
<img src = "http://i.imgur.com/rhkiopM.png">

Remember the mask function we looked at last lesson? Let's try to compare <code>facebook_old</code> with <code>facebook_fixed</code> by using the mask function. Save this as <code>facebook_compared</code> using <code>facebook_old</code> as the parameter of the mask function. Then print to confirm!

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val facebook_compared = facebook_fixed.mask(facebook_old)

for (triplet <- facebook_compared.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is ")
    print(triplet.attr)
    print(" with ")
    println(triplet.dstAttr._1)
}
-->

Interesting. We are left with nothing that resembles facebook_old... Why is that? Let's try the same thing but swap <code>facebook_fixed</code> and <code>facebook_old</code> around. Overwrite <code>facebook_compared</code>.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val facebook_compared = facebook_old.mask(facebook_fixed)

for (triplet <- facebook_compared.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is ")
    print(triplet.attr)
    print(" with ")
    println(triplet.dstAttr._1)
}
-->

Now that's more like it! The factor that kept this relation was that Stan Smith had the same Id as Andrew Smith, Clark Kent had the same Id as the Captain America Fan Page, and that the relation (edge) was pointed in the same direction. Mask restrict the graph to the vertices and edges that are also in another (looking at Vertex Id and Edge Source and Destination) while keeping source attributes.

Now wouldn't it be interesting if we found out what pageRank thinks about our graph (<code>facebook_fixed</code>)? Guess which is #1!

Let's run pageRank with a value of 0.1 and print out the results without saving it to a variable.

Hint: You will need to call 3 function:
1. PageRank
2. Vertices
3. Collect

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
for (rankee <- facebook_fixed.pageRank(0.1).vertices.collect) {
    println(rankee)
}
-->

Interesting! James is #1. Now let's take a look at Neighborhood Aggregation. Neighborhood Aggregation is important because we can pull valuable information from our graph. Such as the number of Friends a person has, or the average age of twitter followers. PageRan for example, repeatedly aggregates properties of neighboring vertices.

For Neighborhood Aggregation, we will be looking at the <code>aggregateMessages</code> function. This function applies a user defined <code>sendMsg</code> function to each <code>EdgeTriplet</code> in the graph and then uses the <code>mergeMsg</code> function to aggregate those messages at their destination vertex.

So <code>aggregateMessages</code> takes in 3 parameters:

- <code>sendMsg</code>
- <code>mergeMsg</code>
- <code>tripletField</code>

<code>sendMsg</code> takes an <code>EdgeContext</code> - which is just a variable representing a triplet (as we have done before to filter values out, create views, and run operators). After defining an <code>EdgeContext</code>, we can use the .sendToDst or .sendToSrc to -... as you may have guess send a message (the parameter of these function) to either destination or source. This will be aggregated by our second parameter - <code>mergeMsg</code>.

<code>mergeMsg</code> will also be represented by arbitrary variables like in <code>sendMsg</code>. However, it will be represented by two variables. Then you define how to merge these multiple messages at the vertex. This is similar to how we defined the merge in <code>groupEdges</code>.

Then the final field which is optional is the <code>tripletField</code>. This field defines what fields of the triplet should be looked at for optimization. If this is not defined, it will be set to <code>TripletFields</code>. All - which is all of the fields. However, if you notice that you only are using the src, edge, or dst field - then you can set it to one or a combination of the field so that when the function is ran, its speed will be optimized because it will not need to run through unnecessary fields.

Another thing to note is that you will need to define a parameter type (after calling the function and before the parameter list). This parameter type will be defining the type for the messages.

<code>aggregateMessages</code> returns a <code>vertexRDD</code> with the aggregated messages attached. This will not include vertices that was not aggregated.

Here's an example:

In [ ]:
val aggregated_vertices = facebook_fixed.aggregateMessages[Int](triplet => triplet.sendToDst(1), (a, b) => (a + b))

In [ ]:
for (aggregated_vertex <- aggregated_vertices.collect) {
    println(aggregated_vertex)
}

So here we created a new variable called <code>aggregated_vertices</code>. We ran <code>aggregatedMessages</code> on <code>facebook_fixed</code>, defining a parameter type of Int (Since we are just sending an Int). Then sent a message to every end of the relationship with an Int of 1. Then defined that we would add the messages together (a + b) if there were multiple messages at one vertex. 

Then we printed it by collecting the result, this was printed to show how many relationships had them as the "Destination Friend" (Just a friend that happened to be the destination of the edge). The first parameter of the printed result is the vertex id and following number will be the count or aggregated result. - Aka how many times that vertex was the "Destination Friend."

Now, finding this information isn't useful at all. Let's try find something useful by finding how many friends every person has in our <code>facebook_fixed</code> graph, not just destination friend. Use the <code>aggregateMessage</code> function and print it out without saving it to a new variable!

Hint: Each relationship is unique.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
for (aggregated_vertex <- facebook_fixed.aggregateMessages[Int](triplet => (triplet.sendToSrc(1), triplet.sendToDst(1)), (a, b) => (a + b)).collect) {
    println(aggregated_vertex)
}
-->

Now you should get a lot more numbers than before! As you can see, we won't be able to do too much with the information provided. However, if there was more information on each vertex, we could for example calculate the average age of its friends. Think about how do you think you would go about doing that.

The next and final function we will talk about is the cache function. It is very simple to use. You call the cache function on a graph you wish to cache. This is important if you are continually using the same graph multiple times. Since the graph will be in memory, it will be improve performance and be faster to compute.

Go ahead try to cache it!

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
facebook_fixed.cache()
-->

Awesome! Now let's take a look at a real example. Let's look at some airport data! Now we will need the help of an ArrayBuffer (which is an Array that is mutable - changeable in a simple explanation) and a HashMap (which is a dictionary - a map of key to values).

You will need to import the following libraries:
- scala.collection.mutable.ArrayBuffer
- collection.mutable.HashMap

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
import scala.collection.mutable.ArrayBuffer
import collection.mutable.HashMap
-->

So this airport data will be in the form of .csv (comma separated value file). We will be using the <code>scala.io.Source.fromFile</code> function to store it in a variable called airport_data.

Download the file here: https://ibm.box.com/s/c805roaenqk9i220q0t48wxgw2adz6ed

Then drag the file onto the work bench. And use the following command to read it in. (Note, the file name must remain the same in the workbench as you downloaded it to work).

In [ ]:
val airport_data = scala.io.Source.fromFile("./airports.csv")

We can go ahead and print out airport_data by running this for loop to print each line to take a look at what we have:

In [ ]:
for (line <- airport_data.getLines()) {
    println(line)
}

That's a lot of data. For a readme of what each column means, follow the link http://openflights.org/data.html#airport. Now we are going to create create an empty ArrayBuffer with the type definition [Long, (String, String, String)] called vertices and a HashMap with the type definition [String,Long] called <code>vertice_map</code>. However, we will be using the {} brackets and using the override command to define a new value (0) for a key that is called, but not present in the HashMap.

In [ ]:
val vertices = ArrayBuffer[(Long, (String, String, String))]()
val vertice_map = new HashMap[String,Long]()  { override def default(key:String) = 0 }

The reason we created a HashMap is so that we can assign the String codes to a Long. We need a Long because when we create an vertex, we need a vertex id of Long, and source and destination ids for edges. So it's important to use a type of HashMap when you deal with relationships involving Strings as their id instead of numbers so that you won't run into problems creating a graph.

Now we will be running through creating the vertices RDD of our graph and mapping the values using the HashMap we created above. I will explain the procedure of the following code and let you see if you can follow with creating the edgesRDD of our graph.

In [ ]:
val airport_data = scala.io.Source.fromFile("./airports.csv")
var counter = 1
for (line <- airport_data.getLines()) {
    val cols = line.split(",")
    if (cols(4) != "\\N" && cols(4) != "") {
        vertice_map += (cols(4) -> counter)
        vertices += ((counter, (cols(4), cols(1), cols(3))))
        counter += 1
    }
}
for (line <- vertices) {
    println(line)
}
val plane_vertexRDD: RDD[(Long, (String, String, String))] = sc.parallelize(vertices)

Now we re-ran the airport_data line because after running the .getLines function, we want to reset the pointer, and an easy way is a re-definition. A counter was then created for indexing, then we used the same for loop as before for printing and split the line in each loop by a comma. Now, the only information that we currently want is in columns 4, 1, and 3. They represent:

Column Index 4 -> Airport Code IATA/FAA
Column Index 1 -> Airport Name
Column Index 3 -> Country Location

More information can be found in the readme link just above!

So, the Airport Code column can contain empty or \N values, so we checked if the line we had didn't have those values, and if it didn't, then we assigned the airport code with the counter to the <code>vertice_map</code>. Then added it, using the counter as the vertex id, incremented it, and appended the relevant information inside of the Array vertices in a similar format to how we did facebook, but a little more info.

Then we just printed how our vertices looked like and created <code>plane_vertexRDD</code> using the parallelize function of <code>sc</code>.

Your turn! Now create a variable called route_data which reads the route data of airplanes similar to how we read in and defined airport_data, except with routes.csv. 

Now just like before, go ahead and download it here: https://ibm.box.com/s/ess4s648mwyod5f3swmcnegy5b67hith

Drag it onto the work bench and make sure the path is correct (Solution is based off of routes.csv being the filename here)

Then you can go ahead and print out the information if you'd like. For more information about these dataset, visit http://openflights.org/data.html#route

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val route_data = scala.io.Source.fromFile("./routes.csv")
-->

Now define another ArrayBuffer called edges with the type class <code>Edge[String]</code>.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val edges = ArrayBuffer[Edge[String]]()
-->

Now, similar as before, use a for loop to run through the lines of route_data (make sure you have re-ran the route_data definition if you ran the .getLines command. This may be if you have failed with incorrect code, or just used it before like in a print statement.)

Then we will split each line by ",". The only important columns we want are:

Column Index 2 -> Airport Source Code<br>
Column Index 4 -> Airport Destination Code<br>
Column Index 0 -> Airline Code<br>

We will then check with an if statement if the <code>vertice_map</code> contains the airport source or destintation code. If they are both registered, then we will append the value to edges with the format - source code, destination code, and airline code similarly as we did before in facebook. Now remember, we changed the return value of the Hashmap to certain number (Hint, Hint) if a value was called and it wasn't inside of the Hashmap.

Then go ahead and print edges after getting your final array (remember you will need to use another for loop).

Finally create a new variable called <code>plane_edgeRDD</code> which is the parallelization of edges.

Note: If you do not see your edges after running it, you want need to re-initialize the route_data.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
for (line <- route_data.getLines()) {
    val cols = line.split(",")
    if (vertice_map(cols(2)) != 0 || vertice_map(cols(4)) != 0) {
        edges += (Edge(vertice_map(cols(2)), vertice_map(cols(4)), cols(0)))
    } 
}
for (line <- edges) {
    println(line)
}
val plane_edgeRDD: RDD[Edge[String]] = sc.parallelize(edges)
-->

Awesome! Now we will create a variable called <code>default_plane_vertex</code> which will be our ... default vertex for our plane graph :). It should be in a similar format - so we will define it as "Location", "Currently", "Unknown".

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
var default_plane_vertex = ("Location", "Currently", "Unknown")
-->

Then we will initiate the Graph function with the parameters - <code>plane_vertexRDD</code>, <code>plane_edgeRDD</code>, and <code>default_plane_vertex</code>. The variable will be called <code>plane_graph</code>.

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val plane_graph = Graph(plane_vertexRDD, plane_edgeRDD, default_plane_vertex)
-->

Now print the view to see the results of our graph!

Remember: You will be printing a view of flights, not of friendship relationships. So make sure the information you plan to print out makes sense together!

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
for (triplet <- plane_graph.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" to ")
    print(triplet.dstAttr._1)
    print(" with Arline ")
    println(triplet.attr)
}
-->

Now, as you can see. We have "Location" a lot. That means that there are a lot of missing vertices that connect to the graph. So we will want to get ride of those edges. Do you remember how to?

Hint: subgraph.

We will create a new graph called plane_graph_fixed using the provided hint to get rid of the relationships with "Location"

Then print to confirm!

In [ ]:
// Type your code here

Double-click __here__ for the solution.

<!-- Your answer is below:
val plane_graph_fixed = plane_graph.subgraph(vpred = (id, user_type) => user_type._1 != "Location")

for (triplet <- plane_graph_fixed.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" to ")
    print(triplet.dstAttr._1)
    print(" with Arline ")
    println(triplet.attr)
}
-->

Awesome! Now we could have also gone further with using groupEdges, but we'll assume that this data provides unique edges so I won't run it this time. (However, that doesn't mean you can't :) ) We also just scratched the surface of this data, we included only the basic information in the graph. We could include more and keep exploring, but I'll leave the venturing to you! Feel free to try to experiment with this data, and more data!

Please check http://spark.apache.org/docs/latest/graphx-programming-guide.html for more resources. This link is for latest Spark version. If you run into problems, Please check your Spark version with the command print(sc.version) to check if there is a problem with matching versions.

Thank you for your time!

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Get IBM Watson Studio free of charge!</h2>
    <p><a href="https://cocl.us/exploring_spark_graphX"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/BD0223EN/Ad/BottomAd.png" width="750" align="center"></a></p>
</div>